# Instalación de paquetes, importación de datos y de Paquetes

In [ ]:
!pip install cplex
!pip install docplex

     |████████████████████████████████| 31.0MB 159kB/s 
     |████████████████████████████████| 614kB 5.6MB/s 
  Created wheel for docplex: filename=docplex-2.19.202-cp36-none-any.whl size=670938 sha256=b992ec69d88b1fac7a0c430878be43c5ec78e45f0520b67546281f7a27f708cd
  Stored in directory: /root/.cache/pip/wheels/33/bb/64/5dd51006e730eb1227b8033d1f9e2246b806d9985a70c90d99
Successfully built docplex


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from docplex.mp.model import Model
import pandas as pd

In [ ]:
excel = pd.ExcelFile('/content/drive/Shareddrives/20201-Logistica Modelo/Listado_plantas_concreto.xlsx')
Demanda_Concreteras= pd.read_excel(excel,"Demanda_Concreteras")
Capacidad_Cementeras=pd.read_excel(excel,"Capacidad_Cementeras")
Capacidad_Cascarilla=pd.read_excel(excel,"Capacidad_Cascarilla")
Capacidad_Ceniza=pd.read_excel(excel,"Capacidad_Ceniza")
Emisiones_tonCem=510.54
Emisiones_cemConc= pd.read_excel(excel,"Cementera_Concretera")
Emisiones_arrConc= pd.read_excel(excel,"Arroceras_Concreteras")
Emisiones_arrPutConsl= pd.read_excel(excel,"Arroceras_Consolidacion")
Emisiones_putConslConc= pd.read_excel(excel,"Consolidacion_Concreteras")

#Escenario Base

Conjuntos

In [ ]:
#Plantas de cemento
M=Capacidad_Cementeras["Cementera"]
#Plantas de concreto
C=Demanda_Concreteras["Concretera"]
#Arcos Cementera Concretera
arcos_MC = [(i,j) for i in M for j in C]

Parámetros

In [ ]:
#Emisiones por tonelada de cemento producida
T=510.54
#Demanda concreteras
Dj = {C[j]: row["Demanda"] for j, row in Demanda_Concreteras.iterrows()}
#Capacidad Cementeras
Pi={M[i]: row["Capacidad"] for i, row in Capacidad_Cementeras.iterrows()}
#Emisiones por tonelada de cemento tranportada en cada arco
Eij={arcos_MC[ij]: row["Emisiones"] for ij, row in Emisiones_cemConc.iterrows()}

Variables de decisión

In [ ]:
mdl=Model('emisiones')
x=mdl.integer_var_dict(arcos_MC,name='x')

Función Objetivo

In [ ]:
mdl.minimize(T*mdl.sum(x[ij] for ij in arcos_MC)+mdl.sum(x[ij]*Eij[ij] for ij in arcos_MC))#función objetivo, i en arcos

Restricciones

In [ ]:
#Se debe suplir la demanda de cada cocretera
for j in C:
  mdl.add_constraint(mdl.sum(x[(i,j)] for i in M)>=Dj[j])#restriccion de demanda
#No se debe superar la capacidad da ninguan cementera
for i in M:
  mdl.add_constraint(mdl.sum(x[i,j] for j in C)<=Pi[i])#restriccion capacidad
#No se pueden mandar cantidades negativas
for ij in arcos_MC:
  mdl.add_constraint(x[ij]>=0)

Solución

In [ ]:
solucion=mdl.solve(log_output=True)
mdl.get_solve_status()
solucion.display()

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 1 time.
MIP Presolve eliminated 28 rows and 0 columns.
Reduced MIP has 11 rows, 28 columns, and 56 nonzeros.
Reduced MIP has 0 binaries, 28 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Found incumbent of value 7785542.072106 after 0.01 sec. (0.09 ticks)
Tried aggregator 1 time.
Reduced MIP has 11 rows, 28 columns, and 56 nonzeros.
Reduced MIP has 0 binaries, 28 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0    

In [ ]:
#Observar el modelo escrito
#print(mdl.export_to_string())

# Escenario 1


Conjuntos


In [ ]:
#Plantas de cemento
M=Capacidad_Cementeras["Cementera"]
#Plantas de concreto
C=Demanda_Concreteras["Concretera"]
#Arroceras
A=Capacidad_Cascarilla["Arrocera"]
#Arcos Cementera Concretera
arcos_MC = [(i,j) for i in M for j in C]
#Arcos Arrocera Concretera
arcos_AC = [(k,j) for j in C for k in A]

Parámetros

In [ ]:
#Emisiones por tonelada de cemento producida
T=510.54
#Demanda concreteras
Dj = {C[j]: row["Demanda"] for j, row in Demanda_Concreteras.iterrows()}
#Capacidad Cementeras
Pi={M[i]: row["Capacidad"] for i, row in Capacidad_Cementeras.iterrows()}
#Capacidad produccion de CCA
Nk={A[k]: row["Ceniza"] for k, row in Capacidad_Ceniza.iterrows()}
#Emisiones por tonelada de cemento tranportada en cada arco
Eij={arcos_MC[ij]: row["Emisiones"] for ij, row in Emisiones_cemConc.iterrows()}
#Emisiones por tonelada de CCA tranportada en cada arco
Fkj={arcos_AC[kj]: row["Emisiones"] for kj, row in Emisiones_arrConc.iterrows()}  

Variables de Decisión

In [ ]:
mdl1=Model('emisiones')
#cemento desde cada cementera a cada cocretera
x=mdl1.integer_var_dict(arcos_MC,name='x')
#CCA desde cada arrocera a cada concretera
y=mdl1.integer_var_dict(arcos_AC,name='y')

Función Objetivo

In [ ]:
mdl1.minimize(T*mdl1.sum(x[ij] for ij in arcos_MC)+mdl1.sum(x[ij]*Eij[ij] for ij in arcos_MC)+mdl1.sum(y[kj]*Fkj[kj] for kj in arcos_AC))

Restricciones

In [ ]:
#Se debe suplir la demanda de cada cocretera
for j in C:
  mdl1.add_constraint(mdl1.sum(x[(i,j)] for i in M)+mdl1.sum(y[(k,j)] for k in A)>=Dj[j])#restriccion de demanda
#No se debe superar la capacidad da ninguna cementera
for i in M:
  mdl1.add_constraint(mdl1.sum(x[i,j] for j in C)<=Pi[i])#restriccion capacidad
#No se debe superar la capacidad de ninguna arrocera
for k in A:
  mdl1.add_constraint(mdl1.sum(y[k,j] for j in C)<=Nk[k])#restriccion capacidad
#CCA no puede superar el 30% del cemento en cada concretera
for j in C:
  mdl1.add_constraint(0.3*mdl1.sum(x[(i,j)] for i in M) >= mdl1.sum(y[(k,j)] for k in A))#restriccion caracteristicas
#No se pueden mandar cantidades negativas
for ij in arcos_MC:
  mdl1.add_constraint(x[ij]>=0)
for kj in arcos_AC:
  mdl1.add_constraint(y[kj]>=0)

Solución

In [ ]:
solucion1=mdl1.solve(log_output=True)
mdl1.get_solve_status()
solucion1.display()

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 1 time.
MIP Presolve eliminated 36 rows and 0 columns.
MIP Presolve modified 5 coefficients.
Reduced MIP has 17 rows, 36 columns, and 108 nonzeros.
Reduced MIP has 0 binaries, 36 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.07 ticks)
Found incumbent of value 4899859.407515 after 0.01 sec. (0.08 ticks)
Tried aggregator 1 time.
Reduced MIP has 17 rows, 36 columns, and 108 nonzeros.
Reduced MIP has 0 binaries, 36 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best B

#Escenario 2

Conjuntos

In [ ]:
#Plantas de cemento
M=Capacidad_Cementeras["Cementera"]
#Plantas de concreto
C=Demanda_Concreteras["Concretera"]
#Arroceras
A=Capacidad_Cascarilla["Arrocera"]
#Centros de consolidación
N=Emisiones_arrPutConsl["Punto_consolidacion"][:4]
#Arcos Cementera Concretera
arcos_MC = [(i,j) for i in M for j in C]
#Arcos Arrocera Puntos de consolidación
arcos_AN = [(k,l) for k in A for l in N]
#Arcos Puntos de consolidación concreteras
arcos_NC = [(l,j) for j in C for l in N]
#Con un solo punto
N=[N[3]]

Parámetros

In [ ]:
#Emisiones por tonelada de cemento producida
T=510.54
#Demanda concreteras
Dj = {C[j]: row["Demanda"] for j, row in Demanda_Concreteras.iterrows()}
#Capacidad Cementeras
Pi={M[i]: row["Capacidad"] for i, row in Capacidad_Cementeras.iterrows()}
#Capacidad produccion de Cascarilla
Mk={A[k]: row["Cascarilla"] for k, row in Capacidad_Cascarilla.iterrows()}
#Emisiones por tonelada de cemento tranportada en cada arco
Eij={arcos_MC[ij]: row["Emisiones"] for ij, row in Emisiones_cemConc.iterrows()}
#Emisiones por tonelada de cascarilla tranportada en cada arco
Fkl={arcos_AN[kl]: row["Emisiones"] for kl, row in Emisiones_arrPutConsl.iterrows()}  
#Emisiones por tonelada de CCA tranportada en cada arco
Hlj={arcos_NC[lj]: row["Emisiones"] for lj, row in Emisiones_putConslConc.iterrows()}  

Variables de decisión

In [ ]:
mdl2=Model('emisiones')
#cemento desde cada cementera a cada cocretera
x=mdl2.integer_var_dict(arcos_MC,name='x')
#cascarilla desde cada arrocera a cada punto de consolidación
y=mdl2.integer_var_dict([(k,l) for k in A for l in N],name='y')
#CCA desde cada punto de consolidación a cada concretera
v=mdl2.integer_var_dict([(l,j) for j in C for l in N],name='v')
#Selección del punto de consolidación
#z=mdl2.binary_var_dict(N,name='z')

Función Objetivo

In [ ]:
mdl2.minimize(T*mdl2.sum(x[ij] for ij in arcos_MC)+mdl2.sum(x[ij]*Eij[ij] for ij in arcos_MC)+mdl2.sum(y[(k,l)]*Fkl[(k,l)] for k in A for l in N)+mdl2.sum(v[(l,j)]*Hlj[(l,j)] for j in C for l in N))

Restricciones

In [ ]:
#Se debe suplir la demanda de cada cocretera
for j in C:
  mdl2.add_constraint(mdl2.sum(x[(i,j)] for i in M)+mdl2.sum(v[(l,j)] for l in N)>=Dj[j])#restriccion de demanda
#No se debe superar la capacidad de ninguna cementera
for i in M:
  mdl2.add_constraint(mdl2.sum(x[i,j] for j in C)<=Pi[i])#restriccion capacidad
#No se debe superar la capacidad de ninguna arrocera
for k in A:
  mdl2.add_constraint(mdl2.sum(y[k,l] for l in N)<=Mk[k])#restriccion capacidad
  #mdl2.add_constraint(mdl2.sum(y[k,l] for l in N) == mdl2.sum(z[l]*y[k,l] for l in N)) 
#CCA no puede superar el 30% del cemento en cada concretera
for j in C:
  mdl2.add_constraint(0.3*mdl2.sum(x[(i,j)] for i in M) >= mdl2.sum(v[(l,j)] for l in N))#restriccion caracteristicas
  #mdl2.add_constraint(mdl2.sum(v[(l,j)] for l in N) == mdl2.sum(z[l]*v[(l,j)] for l in N))#restriccion caracteristicas
#No se pueden mandar cantidades negativas
for i in M:
  for j in C:
    mdl2.add_constraint(x[(i,j)]>=0)
for k in A:
  for l in N:
    mdl2.add_constraint(y[(k,l)]>=0)
for j in C:
  for l in N:
    mdl2.add_constraint(v[(l,j)]>=0)
#No se debe superar la capacidad CCA en el punto de consolidación
for l in N:
  mdl2.add_constraint(mdl2.sum(v[(l,j)] for j in C)<=0.2*mdl2.sum(y[(k,l)] for k in A))
#Solo debe haber un punto de consolidación
#mdl2.add_constraint(mdl2.sum(z) <=1)

Solución

In [ ]:
#print(mdl2.export_to_string())

In [ ]:
solucion2=mdl2.solve(log_output=True)
mdl2.get_solve_status()
solucion2.display()

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 1 time.
MIP Presolve eliminated 36 rows and 0 columns.
MIP Presolve modified 5 coefficients.
Reduced MIP has 16 rows, 34 columns, and 98 nonzeros.
Reduced MIP has 0 binaries, 34 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.06 ticks)
Found incumbent of value 4899859.407515 after 0.01 sec. (0.07 ticks)
Tried aggregator 1 time.
Reduced MIP has 16 rows, 34 columns, and 98 nonzeros.
Reduced MIP has 0 binaries, 34 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bou